# Libraries

In [1]:
pip install imbalanced-learn

In [2]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob #to use for sentiment analysis
from wordcloud import WordCloud

# Understanding and Preprocessing

## Mount it to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Specify the path to the directory where your CSV files are stored
main_directory = '/content/drive/My Drive/datasets/Fakenews_dataset'

# Load PolitiFact CSV files
politifact_mf = pd.read_csv(os.path.join(main_directory, 'politifact_MF.csv'))
politifact_hf = pd.read_csv(os.path.join(main_directory, 'politifact_HF.csv'))
politifact_mr = pd.read_csv(os.path.join(main_directory, 'politifact_MR.csv'))
politifact_hr = pd.read_csv(os.path.join(main_directory, 'politifact_HR.csv'))

# Load GossipCop CSV files
gossipcop_mf = pd.read_csv(os.path.join(main_directory, 'gossipcop_MF.csv'))
gossipcop_hf = pd.read_csv(os.path.join(main_directory, 'gossipcop_HF.csv'))
gossipcop_mr = pd.read_csv(os.path.join(main_directory, 'gossipcop_MR.csv'))
gossipcop_hr = pd.read_csv(os.path.join(main_directory, 'gossipcop_HR.csv'))

## Explore the Data (EDA)

In [5]:
def display_heads(datasets, n=5):
    for name, df in datasets.items():
        print(f"First {n} rows of {name}:")
        print(df.head(n))
        print("\n" + "="*50 + "\n")


# Store datasets in a dictionary
datasets = {
    'PolitiFact MF': politifact_mf,
    'PolitiFact HF': politifact_hf,
    'PolitiFact MR': politifact_mr,
    'PolitiFact HR': politifact_hr,
    'GossipCop MF': gossipcop_mf,
    'GossipCop HF': gossipcop_hf,
    'GossipCop MR': gossipcop_mr,
    'GossipCop HR': gossipcop_hr
}

# Display the first few rows of each dataset
display_heads(datasets)

First 5 rows of PolitiFact MF:
                id                                        description  \
0  politifact11773  Republican attacks on transgendered Americans ...   
1  politifact13827  Whoopi Goldberg is in hot water after comments...   
2  politifact13570  Washington, DC — A former Secret Service agent...   
3  politifact14947  Bill Clinton’s hitman has confessed to more th...   
4  politifact14517  Scott&#8217;s prognosis isn&#8217;t good. (via...   

                                                text  \
0  inia State Representative Mark Cole's proposed...   
1  Whoopi Goldberg has found herself in the middl...   
2  A former Secret Service agent has written a ne...   
3  In what appears to be a major blow to the cred...   
4  In a shocking turn of events, Florida Governor...   

                                               title  
0  Virginia Republican Introduces Controversial B...  
1  Whoopi Goldberg Faces Backlash for Disrespectf...  
2  Former Secret Service Age

In [6]:
def display_info(datasets):
    for name, df in datasets.items():
        print(f"Information about {name}:")
        df.info()
        print("\n" + "="*50 + "\n")

# Display information about each dataset
display_info(datasets)

Information about PolitiFact MF:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           97 non-null     object
 1   description  97 non-null     object
 2   text         97 non-null     object
 3   title        97 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


Information about PolitiFact HF:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           97 non-null     object
 1   description  97 non-null     object
 2   text         97 non-null     object
 3   title        97 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


Information about PolitiFact MR:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 4 columns):
 #   Column      

In [7]:
def display_descriptions(datasets):
    for name, df in datasets.items():
        print(f"Descriptive statistics for {name}:")
        print(df.describe(include='all'))
        print("\n" + "="*50 + "\n")

# Display descriptive statistics for each dataset
display_descriptions(datasets)

Descriptive statistics for PolitiFact MF:
                     id description  \
count                97          97   
unique               97          96   
top     politifact14376     Tribune   
freq                  1           2   

                                                     text  \
count                                                  97   
unique                                                 97   
top     A recent fake news article claimed that former...   
freq                                                    1   

                                                    title  
count                                                  97  
unique                                                 97  
top     Barack Obama Never Tweeted SICK Attack On John...  
freq                                                    1  


Descriptive statistics for PolitiFact HF:
                     id description  \
count                97          97   
unique               97          9

#### Missing values

In [8]:
def display_missing_values(datasets):
    for name, df in datasets.items():
        print(f"Missing values in {name}:")
        missing_values = df.isnull().sum()
        print(missing_values[missing_values > 0])  # Display only columns with missing values
        print("\n" + "="*50 + "\n")

# Display missing values for each dataset
display_missing_values(datasets)

Missing values in PolitiFact MF:
Series([], dtype: int64)


Missing values in PolitiFact HF:
Series([], dtype: int64)


Missing values in PolitiFact MR:
Series([], dtype: int64)


Missing values in PolitiFact HR:
title    1
dtype: int64


Missing values in GossipCop MF:
Series([], dtype: int64)


Missing values in GossipCop HF:
Series([], dtype: int64)


Missing values in GossipCop MR:
Series([], dtype: int64)


Missing values in GossipCop HR:
Series([], dtype: int64)




In [9]:
# Dropping the NA value
politifact_hr.dropna(subset=['title'], inplace=True)

## Combined df

### Feature Engineering

In [10]:
import pandas as pd

# Assuming your datasets are already loaded as DataFrames
# Assign labels and sources based on your description
datasets = {
    'PolitiFact MF': (politifact_mf, 'PolitiFact', 'LLM Generated', 'Fake'),
    'PolitiFact HF': (politifact_hf, 'PolitiFact', 'Human Written', 'Fake'),
    'PolitiFact MR': (politifact_mr, 'PolitiFact', 'LLM Generated', 'Real'),
    'PolitiFact HR': (politifact_hr, 'PolitiFact', 'Human Written', 'Real'),
    'GossipCop MF': (gossipcop_mf, 'GossipCop', 'LLM Generated', 'Fake'),
    'GossipCop HF': (gossipcop_hf, 'GossipCop', 'Human Written', 'Fake'),
    'GossipCop MR': (gossipcop_mr, 'GossipCop', 'LLM Generated', 'Real'),
    'GossipCop HR': (gossipcop_hr, 'GossipCop', 'Human Written', 'Real')
}

# List to hold all dataframes with additional columns
df_list = []

# Iterate over datasets to add the 'platform', 'source', and 'label' columns
for name, (df, platform, source, label) in datasets.items():
    df = df.copy()  # Avoid modifying the original data
    df['platform'] = platform
    df['source'] = source
    df['label'] = label
    df_list.append(df)

# Concatenate all datasets into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the combined DataFrame to verify
print(combined_df.head())
print(combined_df['platform'].value_counts())
print(combined_df['source'].value_counts())
print(combined_df['label'].value_counts())


                id                                        description  \
0  politifact11773  Republican attacks on transgendered Americans ...   
1  politifact13827  Whoopi Goldberg is in hot water after comments...   
2  politifact13570  Washington, DC — A former Secret Service agent...   
3  politifact14947  Bill Clinton’s hitman has confessed to more th...   
4  politifact14517  Scott&#8217;s prognosis isn&#8217;t good. (via...   

                                                text  \
0  inia State Representative Mark Cole's proposed...   
1  Whoopi Goldberg has found herself in the middl...   
2  A former Secret Service agent has written a ne...   
3  In what appears to be a major blow to the cred...   
4  In a shocking turn of events, Florida Governor...   

                                               title    platform  \
0  Virginia Republican Introduces Controversial B...  PolitiFact   
1  Whoopi Goldberg Faces Backlash for Disrespectf...  PolitiFact   
2  Former Secret Ser

In [11]:
combined_df

,id,description,text,title,platform,source,label
0,politifact11773,Republican attacks on transgendered Americans ...,inia State Representative Mark Cole's proposed...,Virginia Republican Introduces Controversial B...,PolitiFact,LLM Generated,Fake
1,politifact13827,Whoopi Goldberg is in hot water after comments...,Whoopi Goldberg has found herself in the middl...,Whoopi Goldberg Faces Backlash for Disrespectf...,PolitiFact,LLM Generated,Fake
2,politifact13570,"Washington, DC — A former Secret Service agent...",A former Secret Service agent has written a ne...,Former Secret Service Agent Exposes Shocking S...,PolitiFact,LLM Generated,Fake
3,politifact14947,Bill Clinton’s hitman has confessed to more th...,In what appears to be a major blow to the cred...,Hannity Issues Retraction on False Story: Bill...,PolitiFact,LLM Generated,Fake
4,politifact14517,Scott&#8217;s prognosis isn&#8217;t good. (via...,"In a shocking turn of events, Florida Governor...",Florida Governor Rick Scott Miraculously Recov...,PolitiFact,LLM Generated,Fake
...,...,...,...,...,...,...,...
21019,gossipcop-875489,From hand-baked clay tiles to LED lights that ...,For free real time breaking news alerts sent s...,The top interior design trends for millennials,GossipCop,Human Written,Real
21020,gossipcop-844263,Gilmore Girls: A Year in the Life made its Net...,Gilmore Girls: A Year in the Life made its Net...,"Gilmore Girls Video: Lauren Graham, Alexis Ble...",GossipCop,Human Written,Real
21021,gossipcop-917467,On Sunday Fox aired “O.J. Simpson: The Lost Co...,Why Is It Airing Now?\n\nAccording to the exec...,"The O.J. Simpson Interview on Fox: Gripping, G...",GossipCop,Human Written,Real
21022,gossipcop-924877,Just when you thought this season of Vanderpum...,Just when you thought this season of Vanderpum...,Kristen Doute and James Kennedy Hooked Up Rumo...,GossipCop,Human Written,Real


In [12]:
combined_df.head()

,id,description,text,title,platform,source,label
0,politifact11773,Republican attacks on transgendered Americans ...,inia State Representative Mark Cole's proposed...,Virginia Republican Introduces Controversial B...,PolitiFact,LLM Generated,Fake
1,politifact13827,Whoopi Goldberg is in hot water after comments...,Whoopi Goldberg has found herself in the middl...,Whoopi Goldberg Faces Backlash for Disrespectf...,PolitiFact,LLM Generated,Fake
2,politifact13570,"Washington, DC — A former Secret Service agent...",A former Secret Service agent has written a ne...,Former Secret Service Agent Exposes Shocking S...,PolitiFact,LLM Generated,Fake
3,politifact14947,Bill Clinton’s hitman has confessed to more th...,In what appears to be a major blow to the cred...,Hannity Issues Retraction on False Story: Bill...,PolitiFact,LLM Generated,Fake
4,politifact14517,Scott&#8217;s prognosis isn&#8217;t good. (via...,"In a shocking turn of events, Florida Governor...",Florida Governor Rick Scott Miraculously Recov...,PolitiFact,LLM Generated,Fake


#### Text Analysis Before Cleaning

##### Readibility score (before)

In [13]:
pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.5 MB/s eta 0:00:00


In [14]:
import textstat

# Calculate readability scores before text cleaning
combined_df['description_readability_before'] = combined_df['description'].apply(textstat.flesch_reading_ease)
combined_df['text_readability_before'] = combined_df['text'].apply(textstat.flesch_reading_ease)
combined_df['title_readability_before'] = combined_df['title'].apply(textstat.flesch_reading_ease)

# Display the first few rows to verify the new columns
print(combined_df[['description', 'description_readability_before', 'text', 'text_readability_before', 'title', 'title_readability_before']].head())


                                         description  \
0  Republican attacks on transgendered Americans ...   
1  Whoopi Goldberg is in hot water after comments...   
2  Washington, DC — A former Secret Service agent...   
3  Bill Clinton’s hitman has confessed to more th...   
4  Scott&#8217;s prognosis isn&#8217;t good. (via...   

   description_readability_before  \
0                           15.65   
1                           65.93   
2                           43.74   
3                           68.30   
4                           14.63   

                                                text  text_readability_before  \
0  inia State Representative Mark Cole's proposed...                    36.12   
1  Whoopi Goldberg has found herself in the middl...                    45.19   
2  A former Secret Service agent has written a ne...                    50.57   
3  In what appears to be a major blow to the cred...                    35.95   
4  In a shocking turn of events, Fl

##### Text Length (before)

In [15]:
# Calculate text length before cleaning
combined_df['title_length_before'] = combined_df['title'].apply(len)
combined_df['description_length_before'] = combined_df['description'].apply(len)
combined_df['text_length_before'] = combined_df['text'].apply(len)

## Text Cleaning

In [16]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure you have downloaded the required NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

def text_preprocessing(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Work on a copy of the text to avoid modifying the original
    processed_text = text.lower()

    # Remove URLs
    processed_text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', processed_text)

    # Remove text enclosed in square brackets
    processed_text = re.sub(r'\[.*?\]', '', processed_text)

    # Remove text enclosed in angle brackets
    processed_text = re.sub(r'<.*?>+', '', processed_text)

    # Remove punctuation characters
    processed_text = re.sub(r'[%s]' % re.escape(string.punctuation), '', processed_text)

    # Remove newline characters
    processed_text = re.sub(r'\n', '', processed_text)

    # Remove substrings of digits surrounded by word characters
    processed_text = re.sub(r'\w*\d\w*', '', processed_text)

    # Remove emails
    processed_text = re.sub(r'@\S+', '', processed_text)

    # Remove numbers
    processed_text = re.sub(r'\b\d+\b', '', processed_text)

    # Remove repeating characters
    processed_text = re.sub(r'(.)\1+', r'\1', processed_text)

    # Tokenize, remove stopwords, stem, and lemmatize
    words = word_tokenize(processed_text)
    words = [word for word in words if word not in stop_words and len(word) > 1]
    words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the processed words back into a single string
    processed_text = " ".join(words)

    return processed_text

### Apply the text_preprocessing function on title, description, and text columns

In [18]:
# Apply the cleaning function on title, description, and text columns
combined_df['cleaned_description'] = combined_df['description'].apply(text_preprocessing)
combined_df['cleaned_text'] = combined_df['text'].apply(text_preprocessing)
combined_df['cleaned_title'] = combined_df['title'].apply(text_preprocessing)

## Text Length (after)

In [19]:
# Calculate text length before cleaning
combined_df['title_length_after'] = combined_df['title'].apply(len)
combined_df['description_length_after'] = combined_df['description'].apply(len)
combined_df['text_length_after'] = combined_df['text'].apply(len)

## Sentiment Analysis

### Sentiment (before)

In [20]:
from textblob import TextBlob

# Function to calculate sentiment polarity
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

combined_df['title_sentiment'] = combined_df['title'].apply(get_sentiment)
combined_df['text_sentiment'] = combined_df['text'].apply(get_sentiment)
combined_df['description_sentiment'] = combined_df['description'].apply(get_sentiment)

### Sentiment (After)

In [21]:
# Apply sentiment analysis on cleaned data
combined_df['cleaned_title_sentiment'] = combined_df['cleaned_title'].apply(get_sentiment)
combined_df['cleaned_text_sentiment'] = combined_df['cleaned_text'].apply(get_sentiment)
combined_df['cleaned_description_sentiment'] = combined_df['cleaned_description'].apply(get_sentiment)


## Readability Score (After)

In [22]:
# Calculate readability scores after text cleaning
combined_df['description_readability_after'] = combined_df['cleaned_description'].apply(textstat.flesch_reading_ease)
combined_df['text_readability_after'] = combined_df['cleaned_text'].apply(textstat.flesch_reading_ease)
combined_df['title_readability_after'] = combined_df['cleaned_title'].apply(textstat.flesch_reading_ease)

In [23]:
combined_df.columns

Index(['id', 'description', 'text', 'title', 'platform', 'source', 'label',
       'description_readability_before', 'text_readability_before',
       'title_readability_before', 'title_length_before',
       'description_length_before', 'text_length_before',
       'cleaned_description', 'cleaned_text', 'cleaned_title',
       'title_length_after', 'description_length_after', 'text_length_after',
       'title_sentiment', 'text_sentiment', 'description_sentiment',
       'cleaned_title_sentiment', 'cleaned_text_sentiment',
       'cleaned_description_sentiment', 'description_readability_after',
       'text_readability_after', 'title_readability_after'],
      dtype='object')

## Model Preparation

#### Scaling (Standardisation)

In [24]:
# Import the StandardScaler class from scikit-learn
from sklearn.preprocessing import StandardScaler

# List of columns in the DataFrame that need to be standardized
columns_to_standardize = [
    'text_length_before',  # Length of text before cleaning
    'description_length_before',  # Length of description before cleaning
    'title_length_before',  # Length of title before cleaning
    'text_length_after',  # Length of text after cleaning
    'description_length_after',  # Length of description after cleaning
    'title_length_after',  # Length of title after cleaning
    'text_readability_before',  # Readability score of text before cleaning
    'description_readability_before',  # Readability score of description before cleaning
    'title_readability_before',  # Readability score of title before cleaning
    'cleaned_text_sentiment',  # Sentiment score of cleaned text
    'cleaned_description_sentiment',  # Sentiment score of cleaned description
    'cleaned_title_sentiment'  # Sentiment score of cleaned title
]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the selected columns and transform them
combined_df[columns_to_standardize] = scaler.fit_transform(combined_df[columns_to_standardize])

# Print the first few rows of the standardized columns to verify the transformation
print(combined_df[columns_to_standardize].head())

   text_length_before  description_length_before  title_length_before  \
0           -0.158926                   2.020107             0.004766   
1           -0.330207                   1.981057            -0.007190   
2           -0.239516                   1.239106            -0.063983   
3           -0.335580                   4.158095             0.034656   
4           -0.269173                  -0.654819             0.013733   

   text_length_after  description_length_after  title_length_after  \
0          -0.158926                  2.020107            0.004766   
1          -0.330207                  1.981057           -0.007190   
2          -0.239516                  1.239106           -0.063983   
3          -0.335580                  4.158095            0.034656   
4          -0.269173                 -0.654819            0.013733   

   text_readability_before  description_readability_before  \
0                -2.340435                       -2.131953   
1               

### Dropping the columns

In [25]:
# List of columns to keep, which includes the scaled columns and the 'source_label' column
columns_to_keep = columns_to_standardize + ['label', 'source', 'cleaned_text']

# Create a new DataFrame with only the selected columns
combined_df1 = combined_df[columns_to_keep]
#gossipcop_df1 = gossipcop_df[columns_to_keep]
#politifact_df1 = politifact_df[columns_to_keep]

# Print the first few rows of the new DataFrame to verify
print(combined_df1.head())
#print(gossipcop_df1.head())
#print(politifact_df1.head())

   text_length_before  description_length_before  title_length_before  \
0           -0.158926                   2.020107             0.004766   
1           -0.330207                   1.981057            -0.007190   
2           -0.239516                   1.239106            -0.063983   
3           -0.335580                   4.158095             0.034656   
4           -0.269173                  -0.654819             0.013733   

   text_length_after  description_length_after  title_length_after  \
0          -0.158926                  2.020107            0.004766   
1          -0.330207                  1.981057           -0.007190   
2          -0.239516                  1.239106           -0.063983   
3          -0.335580                  4.158095            0.034656   
4          -0.269173                 -0.654819            0.013733   

   text_readability_before  description_readability_before  \
0                -2.340435                       -2.131953   
1               

### One hot Encoding (Label and Source Column)

In [26]:
combined_df1.columns

Index(['text_length_before', 'description_length_before',
       'title_length_before', 'text_length_after', 'description_length_after',
       'title_length_after', 'text_readability_before',
       'description_readability_before', 'title_readability_before',
       'cleaned_text_sentiment', 'cleaned_description_sentiment',
       'cleaned_title_sentiment', 'label', 'source', 'cleaned_text'],
      dtype='object')

In [27]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, dtype=int)  # Setting dtype=int to ensure integer output

# Fit and transform both 'label' and 'source' columns
encoded_columns = encoder.fit_transform(combined_df1[['label', 'source']])

# Convert the encoded columns into a DataFrame with proper column names
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['label', 'source']))

# Concatenate the encoded columns back to the original dataframe
combined_df1 = pd.concat([combined_df1, encoded_df], axis=1)

# drop the original 'label' and 'source' columns
combined_df1 = combined_df1.drop(['label', 'source'], axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
combined_df1.columns

Index(['text_length_before', 'description_length_before',
       'title_length_before', 'text_length_after', 'description_length_after',
       'title_length_after', 'text_readability_before',
       'description_readability_before', 'title_readability_before',
       'cleaned_text_sentiment', 'cleaned_description_sentiment',
       'cleaned_title_sentiment', 'cleaned_text', 'label_Fake', 'label_Real',
       'source_Human Written', 'source_LLM Generated'],
      dtype='object')

In [29]:
# Dropping the columns related to the target
combined_df1 = combined_df1.drop(['label_Real','source_Human Written'], axis=1)

In [30]:
combined_df1.columns

Index(['text_length_before', 'description_length_before',
       'title_length_before', 'text_length_after', 'description_length_after',
       'title_length_after', 'text_readability_before',
       'description_readability_before', 'title_readability_before',
       'cleaned_text_sentiment', 'cleaned_description_sentiment',
       'cleaned_title_sentiment', 'cleaned_text', 'label_Fake',
       'source_LLM Generated'],
      dtype='object')

##### Distribution of the target column

In [31]:
class_distribution = combined_df1['label_Fake'].value_counts()
print(class_distribution)

label_Fake
0    12662
1     8362
Name: count, dtype: int64


In [32]:
# Number of unique categories in the 'label_Fake' column
num_unique_categories = combined_df1['label_Fake'].nunique()
print(f"Number of unique categories: {num_unique_categories}")

# Display the unique categories
unique_categories = combined_df1['label_Fake'].unique()
print(f"Unique categories: {unique_categories}")

Number of unique categories: 2
Unique categories: [1 0]


In [33]:
# Count the number of occurrences of each unique value in the 'source_label' column
category_counts = combined_df1['label_Fake'].value_counts()

# Display the counts
print(category_counts)

label_Fake
0    12662
1     8362
Name: count, dtype: int64


In [34]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import pandas as pd

# Separate the features (X) and target (y)
X = combined_df1.drop(columns=['label_Fake'])  # Assuming all other columns are features
y = combined_df1['label_Fake']  # The target column

# Initialize the RandomOverSampler
oversampler = RandomOverSampler(random_state=42)

# Apply the oversampler to the dataset
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Combine the resampled X and y into a new balanced DataFrame
df_balanced = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['label_Fake'])], axis=1)

# Display the first few rows of the balanced dataframe
print(df_balanced.head())

# Check class distribution after oversampling
print("Class distribution after oversampling:", Counter(df_balanced['label_Fake']))


   text_length_before  description_length_before  title_length_before  \
0           -0.158926                   2.020107             0.004766   
1           -0.330207                   1.981057            -0.007190   
2           -0.239516                   1.239106            -0.063983   
3           -0.335580                   4.158095             0.034656   
4           -0.269173                  -0.654819             0.013733   

   text_length_after  description_length_after  title_length_after  \
0          -0.158926                  2.020107            0.004766   
1          -0.330207                  1.981057           -0.007190   
2          -0.239516                  1.239106           -0.063983   
3          -0.335580                  4.158095            0.034656   
4          -0.269173                 -0.654819            0.013733   

   text_readability_before  description_readability_before  \
0                -2.340435                       -2.131953   
1               

In [35]:
# use 30% of the data
sample_df = df_balanced.sample(frac=0.3, random_state=42).reset_index(drop=True)

In [36]:
sample_df.columns

Index(['text_length_before', 'description_length_before',
       'title_length_before', 'text_length_after', 'description_length_after',
       'title_length_after', 'text_readability_before',
       'description_readability_before', 'title_readability_before',
       'cleaned_text_sentiment', 'cleaned_description_sentiment',
       'cleaned_title_sentiment', 'cleaned_text', 'source_LLM Generated',
       'label_Fake'],
      dtype='object')

# The Model

In [37]:
!pip install transformers fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 15.0 MB/s eta 0:00:00


## Data Preparation and Tokenization

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Split into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(
    sample_df[['cleaned_text', 'source_LLM Generated', 'label_Fake']],
    test_size=0.2, random_state=42
)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data in train and test datasets
train_encodings = tokenizer(
    train_df['cleaned_text'].tolist(),
    truncation=True, padding=True, max_length=128
)
test_encodings = tokenizer(
    test_df['cleaned_text'].tolist(),
    truncation=True, padding=True, max_length=128
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Custom Dataset Class

In [50]:
import torch

# Create a custom dataset class
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, llm_source, labels):
        self.encodings = encodings
        self.llm_source = llm_source
        self.labels = labels

    def __getitem__(self, idx):
        # Convert the encodings and features to tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['source_LLM'] = torch.tensor(self.llm_source[idx], dtype=torch.float32)
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Prepare train and test datasets
train_dataset = FakeNewsDataset(
    train_encodings, train_df['source_LLM Generated'].tolist(), train_df['label_Fake'].tolist()
)
test_dataset = FakeNewsDataset(
    test_encodings, test_df['source_LLM Generated'].tolist(), test_df['label_Fake'].tolist()
)


## Multi-Input BERT Model with Dropout Layer

In [51]:
import torch.nn as nn
from transformers import BertModel

class MultiInputBERTModel(nn.Module):
    def __init__(self, num_labels=2):
        super(MultiInputBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc_llm_source = nn.Linear(1, 16)  # Fully connected layer for LLM source
        self.dropout = nn.Dropout(p=0.3)  # Dropout layer to prevent overfitting
        self.fc_combined = nn.Linear(self.bert.config.hidden_size + 16, num_labels)

    def forward(self, input_ids=None, attention_mask=None, source_LLM=None, labels=None):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output  # Get pooled output for classification

        # Process the source_LLM feature
        llm_source_emb = self.fc_llm_source(source_LLM.unsqueeze(1))  # Fully connected LLM source

        # Concatenate BERT embeddings with source_LLM embeddings
        combined_output = torch.cat((pooled_output, llm_source_emb), dim=1)

        # Apply dropout
        combined_output = self.dropout(combined_output)

        # Pass through the final classification layer
        logits = self.fc_combined(combined_output)

        # If labels are provided, calculate loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.fc_combined.out_features), labels.view(-1))

        return (loss, logits) if loss is not None else logits


## Training and Evaluation with Trainer API

In [52]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"      # Evaluate every epoch
)

# Initialize Trainer with the custom model and datasets
model = MultiInputBERTModel(num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Predict on the test dataset
predictions = trainer.predict(test_dataset)
print(predictions)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.463700,0.428225
2,0.404400,0.509229
3,0.317700,0.437147


{'eval_loss': 0.43714746832847595, 'eval_runtime': 2.6077, 'eval_samples_per_second': 582.882, 'eval_steps_per_second': 36.43, 'epoch': 3.0}
PredictionOutput(predictions=array([[-0.8644817,  0.9429889],
       [-2.2515738,  2.639484 ],
       [-1.2837969,  1.7189847],
       ...,
       [ 1.2075019, -1.1966631],
       [-2.1776807,  2.459773 ],
       [-2.0600054,  2.5234256]], dtype=float32), label_ids=array([0, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.43714746832847595, 'test_runtime': 2.7803, 'test_samples_per_second': 546.706, 'test_steps_per_second': 34.169})


## Fairness Metrics Using fairlearn

In [53]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
    true_positive_rate_difference,
    false_positive_rate_difference,
    false_negative_rate_difference
)

# Get predictions and true labels
y_pred = predictions.predictions.argmax(axis=1)
y_true = predictions.label_ids

# Sensitive feature
sensitive_features = test_df['source_LLM Generated']

# Define the evaluation metrics
metrics = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'recall': recall_score,
    'f1': f1_score
}

# Calculate metrics per group using MetricFrame
metric_frame = MetricFrame(
    metrics=metrics,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
)

# Display metrics per group
print("Metrics per group:")
print(metric_frame.by_group)

# Calculate overall metrics
overall_accuracy = accuracy_score(y_true, y_pred)
overall_precision = precision_score(y_true, y_pred, average='weighted')
overall_recall = recall_score(y_true, y_pred, average='weighted')
overall_f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}")

# Calculate group differences
accuracy_group_diff = metric_frame.difference(method='between_groups')['accuracy']
precision_group_diff = metric_frame.difference(method='between_groups')['precision']
recall_group_diff = metric_frame.difference(method='between_groups')['recall']
f1_group_diff = metric_frame.difference(method='between_groups')['f1']

# Print group differences
print(f"Accuracy Group Difference: {accuracy_group_diff:.4f}")
print(f"Precision Group Difference: {precision_group_diff:.4f}")
print(f"Recall Group Difference: {recall_group_diff:.4f}")
print(f"F1 Group Difference: {f1_group_diff:.4f}")

# Fairness metrics
dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features)
print("Demographic Parity Difference:", dp_diff)

eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features)
print("Equalized Odds Difference:", eo_diff)

tpr_diff = true_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features)
print("Equal Opportunity Difference (TPR):", tpr_diff)

fpr_diff = false_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features)
print("False Positive Rate Difference:", fpr_diff)

fnr_diff = false_negative_rate_difference(y_true, y_pred, sensitive_features=sensitive_features)
print("False Negative Rate Difference:", fnr_diff)


Metrics per group:
                      accuracy  precision    recall        f1
source_LLM Generated                                         
0                     0.804087   0.769912  0.754335  0.762044
1                     0.886628   0.891954  0.926014  0.908665
Overall Accuracy: 0.8414
Overall Precision: 0.8415
Overall Recall: 0.8414
Overall F1-Score: 0.8414
Accuracy Group Difference: 0.0825
Precision Group Difference: 0.1220
Recall Group Difference: 0.1717
F1 Group Difference: 0.1466
Demographic Parity Difference: 0.22481551878354206
Equalized Odds Difference: 0.1716790596934623
Equal Opportunity Difference (TPR): 0.1716790596934623
False Positive Rate Difference: 0.014227362430584256
False Negative Rate Difference: 0.17167905969346225


## Adversarial Training Setup

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference, true_positive_rate_difference

In [55]:
# Define the classifier model
class Classifier(nn.Module):
    def __init__(self, num_labels=2):
        super(Classifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

In [56]:
# Define the adversary model
class Adversary(nn.Module):
    def __init__(self):
        super(Adversary, self).__init__()
        self.fc = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, logits):
        return self.sigmoid(self.fc(logits))

In [57]:
# Loss functions
classification_loss_fn = nn.CrossEntropyLoss()
adversary_loss_fn = nn.BCELoss()

# Initialize models and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = Classifier().to(device)
adversary = Adversary().to(device)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=2e-5)
adversary_optimizer = torch.optim.Adam(adversary.parameters(), lr=1e-4)

# Gradual adversarial training
num_epochs = 10
#initial_lambda = 0.1
#lambda_increment = 0.05
initial_lambda = 0.5  # Try increasing this to put more focus on debiasing
lambda_increment = 0.1  # Increase the step size
batch_size = 16

# DataLoader setup
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [58]:
# Initialize variables to track best epoch values
best_accuracy = 0
best_precision = 0
best_recall = 0
best_f1 = 0
best_epoch = 0

In [59]:
# Training loop
for epoch in range(num_epochs):
    classifier.train()
    adversary.train()

    lambda_adversary = initial_lambda + epoch * lambda_increment

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        sensitive_attr = batch['source_LLM'].to(device)

        # Step 1: Train classifier
        classifier_optimizer.zero_grad()
        logits = classifier(input_ids=input_ids, attention_mask=attention_mask)
        classification_loss = classification_loss_fn(logits, labels)

        # Step 2: Train adversary
        adversary_optimizer.zero_grad()
        adversary_logits = adversary(logits.detach())
        adversary_loss = adversary_loss_fn(adversary_logits.squeeze(), sensitive_attr.float())
        adversary_loss.backward()
        adversary_optimizer.step()

        # Step 3: Update classifier to fool adversary
        classifier_optimizer.zero_grad()
        logits = classifier(input_ids=input_ids, attention_mask=attention_mask)
        adversary_logits = adversary(logits)
        adversary_loss_for_classifier = adversary_loss_fn(adversary_logits.squeeze(), sensitive_attr.float())

        total_loss = classification_loss - (lambda_adversary * adversary_loss_for_classifier)
        total_loss.backward()
        classifier_optimizer.step()

    # Evaluation phase: Calculate metrics on test set
    classifier.eval()
    all_preds, all_labels, all_sensitive_attrs = [], [], []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            sensitive_attr = batch['source_LLM'].to(device)

            logits = classifier(input_ids=input_ids, attention_mask=attention_mask)
            preds = logits.argmax(dim=-1)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
            all_sensitive_attrs.append(sensitive_attr.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_sensitive_attrs = torch.cat(all_sensitive_attrs)

    # Calculate metrics
    accuracy = accuracy_score(all_labels.numpy(), all_preds.numpy())
    precision = precision_score(all_labels.numpy(), all_preds.numpy(), average='weighted')
    recall = recall_score(all_labels.numpy(), all_preds.numpy(), average='weighted')
    f1 = f1_score(all_labels.numpy(), all_preds.numpy(), average='weighted')

    # Fairness metrics
    dp_diff = demographic_parity_difference(all_labels.numpy(), all_preds.numpy(), sensitive_features=all_sensitive_attrs.numpy())
    eo_diff = equalized_odds_difference(all_labels.numpy(), all_preds.numpy(), sensitive_features=all_sensitive_attrs.numpy())
    eo_opportunity_diff = true_positive_rate_difference(all_labels.numpy(), all_preds.numpy(), sensitive_features=all_sensitive_attrs.numpy())

    # Calculate metrics per group using MetricFrame
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score
    }
    metric_frame = MetricFrame(
        metrics=metrics,
        y_true=all_labels.numpy(),
        y_pred=all_preds.numpy(),
        sensitive_features=all_sensitive_attrs.numpy()
    )

    accuracy_group_diff = metric_frame.difference(method='between_groups')['accuracy']
    precision_group_diff = metric_frame.difference(method='between_groups')['precision']
    recall_group_diff = metric_frame.difference(method='between_groups')['recall']
    f1_group_diff = metric_frame.difference(method='between_groups')['f1']

    # Track best epoch values
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_epoch = epoch + 1  # Track the 1-based epoch number

    # Print metrics for this epoch
    print(f"Epoch {epoch + 1} | Classification Loss: {classification_loss.item()} | Adversarial Loss: {adversary_loss.item()}")
    print(f"  Demographic Parity: {dp_diff:.4f} | Equalized Odds: {eo_diff:.4f} | Equal Opportunity: {eo_opportunity_diff:.4f}")
    print(f"  Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-Score: {f1:.4f}")
    print(f"  Accuracy Group Difference: {accuracy_group_diff:.4f} | Precision Group Difference: {precision_group_diff:.4f}")
    print(f"  Recall Group Difference: {recall_group_diff:.4f} | F1 Group Difference: {f1_group_diff:.4f}")

Epoch 1 | Classification Loss: 0.6348692774772644 | Adversarial Loss: 4.309541702270508
  Demographic Parity: 0.0643 | Equalized Odds: 0.0889 | Equal Opportunity: 0.0889
  Accuracy: 0.5388 | Precision: 0.7461 | Recall: 0.5388 | F1-Score: 0.4187
  Accuracy Group Difference: 0.1346 | Precision Group Difference: 0.0364
  Recall Group Difference: 0.0889 | F1 Group Difference: 0.1512
Epoch 2 | Classification Loss: 0.6770754456520081 | Adversarial Loss: 3.5913045406341553
  Demographic Parity: 0.3311 | Equalized Odds: 0.3260 | Equal Opportunity: 0.3260
  Accuracy: 0.7724 | Precision: 0.7726 | Recall: 0.7724 | F1-Score: 0.7723
  Accuracy Group Difference: 0.0972 | Precision Group Difference: 0.1115
  Recall Group Difference: 0.3260 | F1 Group Difference: 0.2235
Epoch 3 | Classification Loss: 0.463833749294281 | Adversarial Loss: 5.077078819274902
  Demographic Parity: 0.3592 | Equalized Odds: 0.3442 | Equal Opportunity: 0.3442
  Accuracy: 0.7934 | Precision: 0.7949 | Recall: 0.7934 | F1-Score

In [60]:
# After training, display best epoch and metrics
print(f"Best Epoch: {best_epoch}")
print(f"Best Accuracy: {best_accuracy:.4f}")
print(f"Best Precision: {best_precision:.4f}")
print(f"Best Recall: {best_recall:.4f}")
print(f"Best F1-Score: {best_f1:.4f}")

Best Epoch: 8
Best Accuracy: 0.8105
Best Precision: 0.8118
Best Recall: 0.8105
Best F1-Score: 0.8104
